In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
from scipy import stats

In [2]:
%cd ./repo/operation_goldfish/Matt

C:\Users\fayma\Desktop\NYCDSA\projects\project3\repo\operation_goldfish\Matt


In [3]:
df = pd.read_csv('./../data/Ames_Housing_Price_Data_raw.csv', index_col=0)

In [4]:
df.columns

Index(['PID', 'GrLivArea', 'SalePrice', 'MSSubClass', 'MSZoning',
       'LotFrontage', 'LotArea', 'Street', 'Alley', 'LotShape', 'LandContour',
       'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1',
       'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond',
       'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
       'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF',
       '2ndFlrSF', 'LowQualFinSF', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond

In [5]:
nulls = {}
for col in df.columns:
    nulls[col] = sum(df[f'{col}'].isna()==True)

In [6]:
df.shape

(2580, 81)

In [7]:
nulls

{'PID': 0,
 'GrLivArea': 0,
 'SalePrice': 0,
 'MSSubClass': 0,
 'MSZoning': 0,
 'LotFrontage': 462,
 'LotArea': 0,
 'Street': 0,
 'Alley': 2412,
 'LotShape': 0,
 'LandContour': 0,
 'Utilities': 0,
 'LotConfig': 0,
 'LandSlope': 0,
 'Neighborhood': 0,
 'Condition1': 0,
 'Condition2': 0,
 'BldgType': 0,
 'HouseStyle': 0,
 'OverallQual': 0,
 'OverallCond': 0,
 'YearBuilt': 0,
 'YearRemodAdd': 0,
 'RoofStyle': 0,
 'RoofMatl': 0,
 'Exterior1st': 0,
 'Exterior2nd': 0,
 'MasVnrType': 14,
 'MasVnrArea': 14,
 'ExterQual': 0,
 'ExterCond': 0,
 'Foundation': 0,
 'BsmtQual': 69,
 'BsmtCond': 69,
 'BsmtExposure': 71,
 'BsmtFinType1': 69,
 'BsmtFinSF1': 1,
 'BsmtFinType2': 70,
 'BsmtFinSF2': 1,
 'BsmtUnfSF': 1,
 'TotalBsmtSF': 1,
 'Heating': 0,
 'HeatingQC': 0,
 'CentralAir': 0,
 'Electrical': 1,
 '1stFlrSF': 0,
 '2ndFlrSF': 0,
 'LowQualFinSF': 0,
 'BsmtFullBath': 2,
 'BsmtHalfBath': 2,
 'FullBath': 0,
 'HalfBath': 0,
 'BedroomAbvGr': 0,
 'KitchenAbvGr': 0,
 'KitchenQual': 0,
 'TotRmsAbvGrd': 0,
 'F

In [8]:
lot = df.filter(regex = '^Lot', axis = 1)

In [9]:
lot['SalePrice'] = df['SalePrice']

<ipython-input-9-17bb8ade2b25>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lot['SalePrice'] = df['SalePrice']


In [10]:
lot

,LotFrontage,LotArea,LotShape,LotConfig,SalePrice
1,NaN,7890,Reg,Corner,126000
2,42.0,4235,Reg,Inside,139500
3,60.0,6060,Reg,Inside,124900
4,80.0,8146,Reg,Corner,114000
5,70.0,8400,Reg,Inside,227000
...,...,...,...,...,...
759,NaN,8854,Reg,Inside,121000
760,NaN,13680,IR1,CulDSac,139600
761,82.0,6270,Reg,Inside,145000
762,NaN,8826,Reg,Inside,217500


In [11]:
df = df.reset_index(drop = True)

In [12]:
import numpy as np
import pandas as pd
import plotly.express as px
from scipy import stats
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm as anova

def EDA_plots(df, features = df.columns):
    for feature in features:
        if feature != 'SalePrice' and feature != 'PID':
            print('feature: ',feature)
            scatter = px.scatter(x = df[f'{feature}'], y = df['SalePrice'])
            scatter.update_layout(
                title={
                    'text': f'Scatterplot, {feature} vs SalePrice',
                    'y':0.95,
                    'x':0.5,
                    'xanchor': 'center',
                    'yanchor': 'top'},
                xaxis_title = f'{feature}',
                yaxis_title = 'SalePrice'
            )
            scatter.show()
            hist = px.histogram(x = df[f'{feature}'])
            hist.update_layout(
                title={
                    'text': f'Distribution of {feature}',
                    'y':0.95,
                    'x':0.5,
                    'xanchor': 'center',
                    'yanchor': 'top'},
                xaxis_title = f'{feature}',
                yaxis_title = 'Frequency'
            )
            hist.show()
            box = px.box(x = df[f'{feature}'], y = df['SalePrice'])
            box.update_layout(
                title={
                    'text': f'Boxplot, {feature} vs SalePrice',
                    'y':0.95,
                    'x':0.5,
                    'xanchor': 'center',
                    'yanchor': 'top'},
                xaxis_title = f'{feature}',
                yaxis_title = 'Frequency'
            )
            box.show()
            temp = df[df[f'{feature}'].isna() == False].reset_index(drop = True)
            if type(temp.loc[0, f'{feature}']) != str:
                price_corr = temp[f'{feature}'].corr(temp['SalePrice'])
                print(f'Correlation between {feature} and sale price is {price_corr}')
                linreg = stats.linregress(temp[f'{feature}'], temp['SalePrice'] )
                print(linreg)
                print('r^2 = ',linreg.rvalue**2)
            if type(temp.loc[0, f'{feature}']) == str:
                fit = ols(f'SalePrice ~ C({feature})', data=temp).fit()
                anova_table = anova(fit, typ=2)
                print(anova_table)
        print()

In [13]:
#EDA_plots(df)